In [ ]:
print("Hello QuTech and Q&CE")

### Installation (Ubuntu 18.04)

#### OpenQL
* Install {g++, cmake, swig, python3.5} with sudo apt-get install {name}
* Download Source code (zip) from https://github.com/QE-Lab/OpenQL/releases (here, Release 0.6.0 is used)
* Extract zip and go inside
* pip3 install -e .
* mkdir cbuild
* cd cbuild
* cmake ..
* make

#### Qxelarator
* Download Source code (zip) from https://github.com/QE-Lab/qx-simulator/releases (here, Release 0.2.5 is used)
* Extract zip and go inside Qxelarator folder
* python3 setup.py install --user
* Copy qxelarator folder inside OpenQL main directory

Start this Jupyter Notebook

### Syntax

* Platform(name, config_file)
* Program(name, platform, qubit_count, creg_count=0)
* Kernel(name, platform, qubit_count, creg_count=0)

In [1]:
from openql import openql as ql
import os

def tryInstall():
    config_fn = os.path.abspath('/media/sf_QWorld/Intel/OpenQL-0.6.0/tests/test_cfg_none_simple.json')
    platform = ql.Platform('platform_none', config_fn)
    total_qubits = 1
    prog = ql.Program('p_name', platform, total_qubits)
    k1 = ql.Kernel('QK1',platform, total_qubits)
    k1.gate("h",[0])
    k1.gate("measure",[0])
    prog.add_kernel(k1)
    prog.compile()
    showQasm()

def showQasm():
    file = open("test_output/p_name.qasm","r")
    for line in file:
        print (line,end='')
    file.close()
    
tryInstall()

version 1.0
# this file has been automatically generated by the OpenQL compiler please do not modify it manually.
qubits 1

.QK1
    h q[0]
    measure q[0]


### Qxelerator

Use QX simulator from within OpenQL

Qx does not accept QASM v1.0 yet, so it needs to be converted by removing the version header and square brackets.

In [2]:
import re

def qasmVerConv():
    file = open("test_output/p_name.qasm","r")
    fileopt = open("test_output/oldie.qasm","w")
    header = True
    for line in file:
        if header:
            header = False
        else:
            x = re.sub('\[','', line)
            x = re.sub('\]','', x)
            print (x,end='')
            fileopt.write(x)
    file.close()
    fileopt.close()

from qxelarator import qxelarator

qx = qxelarator.QX()
qx.set('test_output/oldie.qasm')

shots = 100
p_soln = 0

for i in range(shots):
    qx.execute()
    c0 = qx.get_measurement_outcome(0)
    #print('{}'.format(c0))
    if c0 == False:
        p_soln = p_soln+1

print(p_soln)

51


### SciPy Optimizers

The minimize function provides a common interface to unconstrained and constrained minimization algorithms for multivariate scalar functions in scipy.optimize. We will focus on 3 optimizers in the SciPy package:
* Unconstrained minimization: Nelder-Mead
* Bound-Constrained minimization: L-BGFS-B
* Constrained minimization: SLSQP

#### Nelder-Mead

Method Nelder-Mead uses the Simplex algorithm. This algorithm is robust in many applications. However, if numerical computation of derivative can be trusted, other algorithms using the first and/or second derivatives information might be preferred for their better performance in general.

The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. It requires only function evaluations and is a good choice for simple minimization problems. However, because it does not use any gradient evaluations, it may take longer to find the minimum.

1. Nelder, J A, and R Mead. 1965. A Simplex Method for Function Minimization. The Computer Journal 7: 308-13.
2. Wright M H. 1996. Direct search methods: Once scorned, now respectable, in Numerical Analysis 1995: Proceedings of the 1995 Dundee Biennial Conference in Numerical Analysis (Eds. D F Griffiths and G A Watson). Addison Wesley Longman, Harlow, UK. 191-208.

scipy.optimize.minimize(fun, x0, args=(), method='Nelder-Mead', tol=None, callback=None, options={'func': None, 'maxiter': None, 'maxfev': None, 'disp': False, 'return_all': False, 'initial_simplex': None, 'xatol': 0.0001, 'fatol': 0.0001, 'adaptive': False})
* fun: The objective function to be minimized.
* x0: Initial guess.
* args: Extra arguments passed to the objective function and its derivatives (fun, jac and hess functions).
* tol: Tolerance for termination.
* callback: Called after each iteration.
* options:
    * func
    * maxiter: Maximum allowed number of iterations.
    * maxfev: Maximum allowed number of function evaluations.
    * disp: Set to True to print convergence messages.
    * return_all
    * initial_simplex: Initial simplex. If given, overrides x0.
    * xatol: Absolute error in xopt between iterations that is acceptable for convergence.
    * fatol: Absolute error in func(xopt) between iterations that is acceptable for convergence.
    * adaptive: Adapt algorithm parameters to dimensionality of problem. Useful for high-dimensional minimization.
    
Rosenbrock function

$f(x) = \sum_{i=2}^N 100(x_{i+1}-x_i^2)^2+(1-x_i)^2$

The minimum value of this function of N variables is 0 which is achieved when $x_i=1$


In [55]:
from scipy.optimize import minimize

x0 = [1.3, 0.7, 0.8, 1.9, 1.2]

from scipy.optimize import rosen
res = minimize(rosen, x0, method='Nelder-Mead', tol=1e-6)
print(res.x)

def my_rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
my_res = minimize(my_rosen, x0, method='Nelder-Mead', options={'xtol':1e-8, 'disp':True})
print(my_res.x)

[1.00000002 1.00000002 1.00000007 1.00000015 1.00000028]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571
[1. 1. 1. 1. 1.]


### VQE

Variational-Quantum-Eigensolver algorithm: The main components of the VQE algorithm are a minimizer function for performing the functional minimization, a function that takes a vector of parameters and returns a pyQuil program, and a Hamiltonian of which to calculate the expectation value.

In [9]:
###################################################################################################################

from scipy.optimize import minimize
import re
from qxelarator import qxelarator
import numpy as np

class VQE(object):
    
    def __init__(self):
        self.minimizer = minimize
        self.minimizer_kwargs = {'method':'Nelder-Mead', 'options':{'ftol':1.0e-6, 'xtol':1.0e-6, 'disp':True}}
    
    def vqe_run(self, ansatz, h, x0):
        
        """
        args:
            ansatz: variational functional closure in cQASM
            h: hamiltonian
            x0: initial parameters for generating the function of the functional
        return:
            x: set of ansats parameters
            fun: scalar value of the objective function
        """
        t_name = "test_output/"+ansatz+".qasm"
        p_name = "test_output/"+ansatz+"_try"+".qasm"
                
        def objective_func(x):
            add_param(x) # If parameterised program construct not available, define new program here            
            return expectation(h)
        
        def add_param(x):
            template = open(t_name,"r")
            prog = open(p_name,"w")
            param_ctr = 0
            for line in template:
                if re.search('\*',line):
                    line_new = re.sub('\*',str(x[param_ctr]), line)
                    param_ctr += 1
                    prog.write(line_new)
                else:
                    prog.write(line)
            template.close()
            prog.close()     
            
        def expectation(h):
            # We will not use the wavefunction (display command) as is not possible in a real QC
            # E = <wf|H|wf> = real(dot(transjugate(wf),dot(H,wf))) 
            qx = qxelarator.QX()
            qx.set(p_name)
            shots = 1000
            p0 = 0
            for i in range(shots):
                qx.execute()
                c0 = qx.get_measurement_outcome(0)
                if c0 == False:
                    p0 = p0+1
            E = (p0/shots)**2 - ((shots-p0)/shots)**2
            return E
        
        args = [objective_func, x0]
        return self.minimizer(*args, **self.minimizer_kwargs)

###################################################################################################################
    
import math

h = np.array([[1,0],[0,-1]]) # Sigma_z as Hamiltonian             
v = VQE()

r = v.vqe_run("vqe",h,[math.pi/2]) # math.pi = -1.0
print(r.status, r.fun, r.x)

'''
# LAST GOOD RESULT

# h = np.array([[1,0],[0,-1]])
# r = v.vqe_run("vqe",h,[math.pi/2])

Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 25
         Function evaluations: 66
0 -1.0 [3.14159265]
'''

"""
KNOWN ISSUES

* Does not work for initial angle of 0
* Does not work for H bigger than 2x2
"""

Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 25
         Function evaluations: 66
0 -1.0 [3.14159265]


'\nKNOWN ISSUES\n\n* Does not work for initial angle of 0\n* Does not work for H bigger than 2x2\n'

### QAOA

Quantum Approximate Optimization Algorithm

In [3]:
from scipy.optimize import minimize
import re
from qxelarator import qxelarator
from functools import reduce
import numpy as np

class VQE(object):
    
    def __init__(self):
        self.minimizer = minimize
        self.minimizer_kwargs = {'method':'Nelder-Mead', 'options':{'maxiter':200, 'ftol':1.0e-8, 'xtol':1.0e-8, 'disp':True}}
    
    def vqe_run(self, ansatz, h, steps, x0, aid, cfs):
        
        """
        args:
            ansatz: variational functional closure in cQASM
            h: hamiltonian
            x0: initial parameters for generating the function of the functional
        return:
            x: set of ansats parameters
            fun: scalar value of the objective function
        """
        t_name = "test_output/"+ansatz+".qasm"
        p_name = "test_output/"+ansatz+"_try"+".qasm"
                
        def objective_func(x):
            add_param(x) # If parameterised program construct not available, define new program here            
            return expectation(h)
        
        def add_param(x):
            template = open(t_name,"r")
            prog = open(p_name,"w")
            param_ctr = 0
            s = 0
            param_max = len(cfs)
            for line in template:
                if re.search('\*',line):
                    if aid[param_ctr] == 0: # beta replacer
                        theta = x[s]
                    else: # gamma replacer
                        theta = x[s+steps]
                    line_new = re.sub('\*',str(theta*cfs[param_ctr]), line)
                    param_ctr += 1
                    if param_ctr == param_max:
                        param_ctr = 0
                        s += 1
                    prog.write(line_new)
                else:
                    prog.write(line)
            template.close()
            prog.close()     
            
        def expectation(h):
            # We will not use the wavefunction (display command) as is not possible in a real QC
            # E = <wf|H|wf> = real(dot(transjugate(wf),dot(H,wf))) 
            
            # WATSON: correct this for n-qubits
            qx = qxelarator.QX()
            qx.set(p_name)
            shots = 1000
            p0 = 0
            for i in range(shots):
                qx.execute()
                c0 = qx.get_measurement_outcome(0)
                if c0 == False:
                    p0 = p0+1
            E = (p0/shots)**2 - ((shots-p0)/shots)**2
            return E
        
        args = [objective_func, x0]
        return self.minimizer(*args, **self.minimizer_kwargs)


class QAOA(object):      
    def get_angles(self, qubits, steps, betas, gammas, ham, ang_id, coeffs):
        # Finds optimal angles with the quantum variational eigensolver method.
        t_name = "test_output/graph.qasm"
        tv_name = "test_output/qaoa.qasm"
        p_name = "test_output/qaoa_try.qasm"

        def make_qaoa():
            cfs = []
            # Make VQE ansatz template from QAOA ansatz
            prog = open(tv_name,"w")
            prog.write("qubits "+str(qubits)+"\n")
            # Reference state preparation
            for i in range(0,qubits):
                prog.write("h q"+str(i)+"\n")
            # Repeat ansatz for specified steps
            for i in range(0,steps):
                template = open(t_name,"r")
                for line in template:
                    prog.write(line)
                template.close()
                cfs = np.hstack((cfs,coeffs))
            prog.close()
            return cfs
            
        full_coeffs = make_qaoa()
        #H_cost = []
        angles = np.hstack((betas, gammas)) # A concatenated list of angles [betas]+[gammas]
        
        v = VQE()
        result = v.vqe_run("qaoa", ham, steps, angles, ang_id, coeffs) # VQE for PauliTerm Hamiltonian and coefficients       
        return result
        
    def probabilities(ang):
        # Computes the probability of each state given a particular set of angles.
        prog = "test_output/qaoa_try.qasm"
        probs = []
        # RUN AND MEASURE ALL n QUBITS, TO DETERINE PROBABILITY OF ALL 2^n STATES
        return probs
        
        
    #def get_string():
        # Compute the most probable string.
        
###################################################################################################################

import networkx as nx

def graph_to_pqasm(g):
    # Specific for Max-Cut Hamiltonian
    # PauliTerm to Gates concept from rigetti/pyquil/pyquil/paulis.py
    coeffs = [] # Weights for the angle parameter for each gate
    angle_id = []
    sZ = np.array([[1,0],[0,-1]])
    sX = np.array([[0,1],[1,0]])
    I = np.eye(2)
    H_cost = np.kron(I,np.kron(I,I))
    H_cost = np.dot(np.kron(I,np.kron(I,sZ)),H_cost)
    H_cost = np.dot(np.kron(I,np.kron(sZ,I)),H_cost)
    H_cost = np.dot(np.kron(I,np.kron(I,sX)),H_cost)
    H_cost = np.dot(np.kron(I,np.kron(sZ,I)),H_cost)
    H_cost = np.dot(np.kron(sZ,np.kron(I,I)),H_cost)
    H_cost = np.dot(np.kron(I,np.kron(sX,I)),H_cost)
    #print(H_cost)
    t_name = "test_output/graph.qasm"
    ansatz = open(t_name,"w")
    for i,j in g.edges():
        # 0.5*Z_i*Z_j
        ansatz.write("cnot q"+str(i)+",q"+str(j)+"\n")
        ansatz.write("rz q"+str(i)+",*\n")
        coeffs.append(2*0.5)
        angle_id.append(0) # beta
        ansatz.write("cnot q"+str(i)+",q"+str(j)+"\n")
        # -0.5*I_0
        ansatz.write("x q"+str(0)+"\n")
        ansatz.write("rz q"+str(0)+",*\n")
        coeffs.append(-1*0.5)
        angle_id.append(0) # beta
        ansatz.write("x q"+str(0)+"\n")
        ansatz.write("rz q"+str(0)+",*\n")
        coeffs.append(-1*0.5)
        angle_id.append(0) # beta
    for i in g.nodes():
        # -X_i
        ansatz.write("h q"+str(i)+"\n")
        ansatz.write("rz q"+str(i)+",*\n")
        coeffs.append(2*-1)
        angle_id.append(1) # gamma
        ansatz.write("h q"+str(i)+"\n")
    ansatz.close()
    return H_cost, coeffs, angle_id
    
###################################################################################################################

# Barbell graph
g = nx.Graph()
g.add_edge(0,1)
g.add_edge(1,2)
hc, coeffs, aid = graph_to_pqasm(g)

steps = 2
qb = len(g.nodes()) # Number of qubits
b = np.random.uniform(0, np.pi, steps) # Initial beta angle parameters of cost Hamiltonian
g = np.random.uniform(0, 2*np.pi, steps) # Initial gamma angle parameters of driving/mixing Hamiltonian

#print(qb,steps,b,g,hc,aid,coeffs)

qaoa_obj = QAOA()

r = qaoa_obj.get_angles(qb,steps,b,g,hc,aid,coeffs)
print(r.status, r.fun, r.x)
'''
Optimization terminated successfully.
         Current function value: 1.000000
         Iterations: 25
         Function evaluations: 149
(array([2.32105514, 2.0138622 ]), array([2.20695693, 1.86485137]))
'''

# The last qaoa_try will have the optimal angles
probs = qaoa_obj.probabilities()
#print(probs)

[]
